In [4]:
import coffea
import coffea.hist as hist
from coffea.nanoevents.methods import vector
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np

from pyjet import cluster
import awkward as ak
import sys
sys.path.insert(0,"../")

from coffea.nanoevents import NanoEventsFactory, BaseSchema

from HNLprocessor import util
from HNLprocessor.HNLproc_4 import delta_r_pairs

/uscms/home/kkwok/work/LLP/CMSSW_10_6_20/src/llp_analyzer/coffeaenv/lib/python3.7/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


# Load events with uproot/ NanoEventsFactory

In [1]:
def getLZDF(f,nEvents=-1,treename="MuonSystem"):
    events_raw = uproot.open(f)[treename]
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    events = events[start:stop]
    return events

def nanoFact(f,treePath="MuonSystem"):
    return NanoEventsFactory.from_root(f,treepath=treePath,schemaclass=BaseSchema).events()

In [13]:
## Method 1
events = NanoEventsFactory.from_root("/uscms/home/kkwok/lpclonglived/HNL/skim_Dec2/HNL_electronType_mHNL4p0_pl1000/HeavyNeutralLepton_Tree_0.root",
                                     treepath="MuonSystem",
                                     schemaclass=BaseSchema,
                                     metadata={"dataset":"HNL" } ## need dataset name for histograms
                                    ).events()
# method 2
# events = uproot.lazy("/uscms/home/kkwok/lpclonglived/HNL/skim/HNL_electronType_mHNL4p0_pl1000/HeavyNeutralLepton_Tree_0.root:MuonSystem")
## method 3
# events_mu = uproot.lazy("/uscms/home/kkwok/lpclonglived/HNL/skim/HNL_muonType_mHNL4p0_pl1000/HeavyNeutralLepton_Tree_1*.root:MuonSystem")



In [14]:
events

<NanoEventsArray [{runNum: 1, lumiSec: 4957, ... -999]}] type='15931 * event'>

# Import processor

## Re-run this cell to reload the processor after editing

In [7]:
import importlib
import HNLprocessor.HNLproc_4
importlib.reload(HNLprocessor.HNLproc_4)


<module 'HNLprocessor.HNLproc_4' from '../HNLprocessor/HNLproc_4.py'>

# Construct and run the processor

In [9]:
from HNLprocessor.HNLproc_4 import MyProcessor

In [11]:
isElectronChannel=True
saveSkim=False
debug=False

proc = MyProcessor(isElectronChannel)


In [15]:
# run the processor
output = proc.process(events)

In [16]:
## all the histograms
output

{'cutflow': <Hist (dataset,region,cutflow) instance at 0x7f447894f750>,
 'nCluster': <Hist (dataset,region,nCluster) instance at 0x7f447894f7d0>,
 'nCluster_dt': <Hist (dataset,region,nCluster) instance at 0x7f447894f850>,
 'nCluster_n-1': <Hist (dataset,nCluster) instance at 0x7f447894f8d0>,
 'accept': <Hist (dataset,gLLP_csc,gLLP_dt) instance at 0x7f447894f950>,
 'ClusterSize': <Hist (dataset,region,ClusterSize) instance at 0x7f447894f9d0>,
 'ClusterTime': <Hist (dataset,region,ClusterTime) instance at 0x7f447894fa50>,
 'ClusterEta': <Hist (dataset,region,ClusterEta) instance at 0x7f447894fad0>,
 'ClusterAvgStation10': <Hist (dataset,region,ClusterAvgStation10) instance at 0x7f447894fb50>,
 'ClusterNStation10': <Hist (dataset,region,ClusterNStation10) instance at 0x7f447894fbd0>,
 'ClusterME11_12': <Hist (dataset,region,ClusterME11_12) instance at 0x7f447894fc50>,
 'ClusterTimeSpread': <Hist (dataset,region,ClusterTimeSpread) instance at 0x7f447894fcd0>,
 'ClusterJetVetoPt': <Hist (d

# Build the objects used in processor

In [20]:
from  HNLprocessor.HNLproc_4 import MyProcessor,buildMask,delta_r_pairs

llp  = proc.buildLLP(events)
muons = util.pack(events,"muon")
good_lep,ele,tight_muons = proc.buildGoodLeptons(events)

cluster = proc.buildCSCcluster(events,good_lep)
dt_cluster = proc.buildDTcluster(events,good_lep)

clusterMasks = proc.selectCSCcluster(cluster,events)
dt_clusterMasks = proc.selectDTcluster(dt_cluster,events)


In [24]:
preselections = ['trigger_ele','MET',"METfilters",'good_lepton']

m =   proc.buildSelectionMasks(events,good_lep,cluster,clusterMasks,dt_cluster,dt_clusterMasks) 

pre_selection = buildMask(m,preselections)
